# Predicting Satisfiability of SAT-3 Problems
## Solving the Classification Task using a ***Long Short-Term Memory (LSTM)*** 

A *satisfiability problem* (SAT problem) is a family of problems, where given a Boolean expression written using only the logical connectives AND, OR, NOT, variables and parentheses, we examine if there is some truth-assignment to the variables that will make the entire expression true. A **SAT-3** problem is a special case of SAT problem, where Boolean expression should have very strict form : the conjunctive normal form (CNF). A CNF is a conjunction of one or more clauses, where a clause is a disjunction of literals; otherwise put, it is an AND of ORs. In the SAT-3 problem, every disjunction consists of 3 literals. An example of a SAT-3 problem instance is the following:
$$(x_1 \vee x_2 \vee \neg x_4) \wedge (\neg x_3 \vee x_4 \vee x_2)$$

The SAT-3 problem is NP-complete and many solvers have been developed in order to prove the (un)satisfiability of the problems and point a truth assignment of the variables. However, not many large instances can be solved by traditional solvers and, thus, some researchers have come up with the idea to use Machine Learning methods in order to solve those problems, given that with the models we cannot have a guaranteed accuracy, but if the model generalizes well, then we would be able to solve some difficult problem-instances that the state-of-the-art solvers cannot.<br>

In this assignment we will try predicting the satisfiability of the instances using a *Graph Transformer Network* and a *Long Short-Term Memory network (LSTM)*. In this notebook will be explained the process of using a LSTM.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import random
__counter__ = random.randint(0,2e9)

from IPython.display import HTML, display

In [3]:
import os, shutil
import json

from tuning import tune_parameters
from data_loader import dataset_processing
from train import training, testing

Device is: cuda:0


### Training and Evaluation process

In order to train the model, a *training set* is used alongside with a *validation set*. The latter is used to measure the performance of the model. The training and validation set were extracted from the 80% (60%-20%) of the data and the rest 20% was kept to evaluate the model after its training (*test set*).  

**Create the dataset**: The dataset is created from the raw data in such a format that it can be loader by the *DataLoader* module of torch.utils.data. <br>

In this case, the data was treated as *timeseries* with sequence length equal to 2. In order for that to be able to happen, some data augmentation was performed. Specifically, as noted by the dataset documentation (<a href="https://www.cs.ubc.ca/~hoos/SATLIB/Benchmarks/SAT/RND3SAT/descr.html">Dataset Documentation</a>), for all problem instances there is a clause number-threshold after which the problem is highly unlikely to be satisfiable. So, before this threshold, the problem is most likely to be satisfiable and, for that reason, it is possible to augment the dataset in two ways :
<ol>
    <li>For each instance provide a satisfiable smaller version of it (the instance below the threshold).</li>
    <li>For each unsatisfiable instance provide another unsatisfiable instance (the instance just above the threshold).</li>
</ol>

In [4]:
pos_weight = dataset_processing()

print(f'pos_weight = {pos_weight}')

Start the data processing...

Satisfiable series of sequence-length 2   : 3701
Unsatisfiable series of sequence-length 2 : 5400

Ratio of SAT   : 0.4067
Ratio of UNSAT : 0.5933

Processing completed.
pos_weight = 1.4590651175358011


It should be noted that the LSTM model needs about 300 times more data that its parameters and in our case we have 970561 parameters and 14092 input data. **Thus, due to the amount of data, even after the augmentation, the model is not expected to reach high values in the evaluation metrics**. 

**Tune parameters**: Tune the parameters of the model (LSTM itself) as well as parameters regarding the training process e.g. *batch-size*, *weight decay* etc. More specidically, the parameters that are tuned are the following.
<ol><li>Model parameters:</li>
    <ul> 
        <li>Number of layers</li>
        <li>Dropout rate</li>
        <li>Hidden Units</li><br>
    </ul>
    <li>Training parameters:</li>
    <ul> 
        <li>Batch size used</li>
        <li>Learning rate</li>
        <li>Weight decay</li>
    </ul>
</ol>

The parameters of the final model are the ones that result in the minimum ***validation error***. <br>
Also, ***early stopping*** is used in order to avoid *overfitting*.

In [5]:
# Tune parameters
best_parameters = tune_parameters(pos_weight=pos_weight)

# Show best parameters
print(f'\nBest hyperparameters were: {best_parameters}\n')
# Store best parameters
with open('./best_parameters_same_sets.txt', 'w') as f:
    f.write(json.dumps(best_parameters))

f.close()


Test number 1 | Start testing new parameter-combination...

Dataset loading...
Dataset loading completed

Model loading...
Model loading completed

Number of model parameters: 102889

EPOCH | 0
Training Loss   : 0.8254
Validation Loss : 0.8388

EPOCH | 1
Training Loss   : 0.8221
Validation Loss : 0.8396

EPOCH | 2
Training Loss   : 0.8158
Validation Loss : 0.8368

EPOCH | 3
Training Loss   : 0.8074
Validation Loss : 0.8300

EPOCH | 4
Training Loss   : 0.8004
Validation Loss : 0.8406

EPOCH | 5
Training Loss   : 0.7858
Validation Loss : 0.8408

EPOCH | 6
Training Loss   : 0.7719
Validation Loss : 0.9042

EPOCH | 7
Training Loss   : 0.7579
Validation Loss : 0.8618

EPOCH | 8
Training Loss   : 0.7390
Validation Loss : 0.9206

EPOCH | 9
Training Loss   : 0.7164
Validation Loss : 0.8895

EPOCH | 10
Training Loss   : 0.6864
Validation Loss : 0.9627

EPOCH | 11
Early stopping activated, with training and validation loss difference: 0.0133

Test number 2 | Start testing new parameter-combinat

Training Loss   : 0.8214
Validation Loss : 0.8463

EPOCH | 2
Training Loss   : 0.8203
Validation Loss : 0.8359

EPOCH | 3
Training Loss   : 0.8088
Validation Loss : 0.8454

EPOCH | 4
Training Loss   : 0.7988
Validation Loss : 0.8401

EPOCH | 5
Training Loss   : 0.7860
Validation Loss : 0.8631

EPOCH | 6
Training Loss   : 0.7733
Validation Loss : 0.8720

EPOCH | 7
Training Loss   : 0.7632
Validation Loss : 0.8916

EPOCH | 8
Training Loss   : 0.7446
Validation Loss : 0.8501

EPOCH | 9
Training Loss   : 0.7174
Validation Loss : 0.8964

EPOCH | 10
Training Loss   : 0.6939
Validation Loss : 0.8807

EPOCH | 11
Early stopping activated, with training and validation loss difference: 0.0136

Test number 10 | Start testing new parameter-combination...

Dataset loading...
Dataset loading completed

Model loading...
Model loading completed

Number of model parameters: 105193

EPOCH | 0
Training Loss   : 0.8278
Validation Loss : 0.8414

EPOCH | 1
Training Loss   : 0.8214
Validation Loss : 0.8463

E

Training Loss   : 0.6615
Validation Loss : 0.9393

EPOCH | 12
Early stopping activated, with training and validation loss difference: 0.0266

Test number 18 | Start testing new parameter-combination...

Dataset loading...
Dataset loading completed

Model loading...
Model loading completed

Number of model parameters: 210641

EPOCH | 0
Training Loss   : 0.8270
Validation Loss : 0.8587

EPOCH | 1
Training Loss   : 0.8240
Validation Loss : 0.8506

EPOCH | 2
Training Loss   : 0.8170
Validation Loss : 0.8449

EPOCH | 3
Training Loss   : 0.8120
Validation Loss : 0.8800

EPOCH | 4
Training Loss   : 0.8050
Validation Loss : 0.8610

EPOCH | 5
Training Loss   : 0.7874
Validation Loss : 0.8603

EPOCH | 6
Training Loss   : 0.7774
Validation Loss : 0.8680

EPOCH | 7
Training Loss   : 0.7609
Validation Loss : 0.8629

EPOCH | 8
Training Loss   : 0.7411
Validation Loss : 0.8800

EPOCH | 9
Training Loss   : 0.7202
Validation Loss : 0.8691

EPOCH | 10
Training Loss   : 0.6902
Validation Loss : 0.8864

E

New best parameters found!


Test number 26 | Start testing new parameter-combination...

Dataset loading...
Dataset loading completed

Model loading...
Model loading completed

Number of model parameters: 414625

EPOCH | 0
Training Loss   : 0.8279
Validation Loss : 0.8824

EPOCH | 1
Training Loss   : 0.8267
Validation Loss : 0.9247

EPOCH | 2
Training Loss   : 0.8188
Validation Loss : 0.8695

EPOCH | 3
Training Loss   : 0.8132
Validation Loss : 0.8670

EPOCH | 4
Training Loss   : 0.8113
Validation Loss : 0.8705

EPOCH | 5
Training Loss   : 0.8025
Validation Loss : 0.8510

EPOCH | 6
Training Loss   : 0.7873
Validation Loss : 0.8598

EPOCH | 7
Training Loss   : 0.7794
Validation Loss : 0.8590

EPOCH | 8
Training Loss   : 0.7558
Validation Loss : 0.8626

EPOCH | 9
Training Loss   : 0.7412
Validation Loss : 0.9003

EPOCH | 10
Training Loss   : 0.7115
Validation Loss : 0.8931

EPOCH | 11
Training Loss   : 0.6807
Validation Loss : 0.9478

EPOCH | 12
Training Loss   : 0.6327
Validation Loss 

Training Loss   : 0.7753
Validation Loss : 0.8674

EPOCH | 8
Training Loss   : 0.7575
Validation Loss : 0.8834

EPOCH | 9
Training Loss   : 0.7379
Validation Loss : 0.8879

EPOCH | 10
Training Loss   : 0.7051
Validation Loss : 0.9225

EPOCH | 11
Training Loss   : 0.6764
Validation Loss : 0.9349

EPOCH | 12
Early stopping activated, with training and validation loss difference: 0.0321

Test number 34 | Start testing new parameter-combination...

Dataset loading...
Dataset loading completed

Model loading...
Model loading completed

Number of model parameters: 448417

EPOCH | 0
Training Loss   : 0.8371
Validation Loss : 0.8848

EPOCH | 1
Training Loss   : 0.8294
Validation Loss : 0.8615

EPOCH | 2
Training Loss   : 0.8190
Validation Loss : 0.8952

EPOCH | 3
Training Loss   : 0.8150
Validation Loss : 0.8776

EPOCH | 4
Training Loss   : 0.8057
Validation Loss : 0.8680

EPOCH | 5
Training Loss   : 0.7979
Validation Loss : 0.8677

EPOCH | 6
Training Loss   : 0.7893
Validation Loss : 0.8676



...

Training Loss   : 0.0127
Validation Loss : 1.5281

EPOCH | 3
Training Loss   : 0.0065
Validation Loss : 1.6938

EPOCH | 4
Training Loss   : 0.0042
Validation Loss : 1.8269

EPOCH | 5
Training Loss   : 0.0035
Validation Loss : 1.9767

EPOCH | 6
Training Loss   : 0.0593
Validation Loss : 1.9824

EPOCH | 7
Training Loss   : 0.0125
Validation Loss : 2.1114

EPOCH | 8
Training Loss   : 0.0037
Validation Loss : 2.2045

EPOCH | 9
Training Loss   : 0.0026
Validation Loss : 2.2809

EPOCH | 10
Training Loss   : 0.0020
Validation Loss : 2.3447

EPOCH | 11
Early stopping activated, with training and validation loss difference: 0.3572

Test number 51 | Start testing new parameter-combination...

Dataset loading...
Dataset loading completed

Model loading...
Model loading completed

Number of model parameters: 414625

EPOCH | 0
Training Loss   : 0.6407
Validation Loss : 0.9990

EPOCH | 1
Training Loss   : 0.1306
Validation Loss : 1.4058

EPOCH | 2
Training Loss   : 0.0372
Validation Loss : 1.6658

E

Training Loss   : 0.3919
Validation Loss : 1.4561

EPOCH | 8
Training Loss   : 0.3447
Validation Loss : 1.7250

EPOCH | 9
Training Loss   : 0.2728
Validation Loss : 1.7091

EPOCH | 10
Training Loss   : 0.2278
Validation Loss : 1.8149

EPOCH | 11
Early stopping activated, with training and validation loss difference: 0.1065

Test number 59 | Start testing new parameter-combination...

Dataset loading...
Dataset loading completed

Model loading...
Model loading completed

Number of model parameters: 414625

EPOCH | 0
Training Loss   : 0.8592
Validation Loss : 0.8959

EPOCH | 1
Training Loss   : 0.8411
Validation Loss : 0.9062

EPOCH | 2
Training Loss   : 0.8300
Validation Loss : 0.8895

EPOCH | 3
Training Loss   : 0.8142
Validation Loss : 0.8680

EPOCH | 4
Training Loss   : 0.8059
Validation Loss : 0.8637

EPOCH | 5
Training Loss   : 0.7777
Validation Loss : 0.9202

EPOCH | 6
Training Loss   : 0.7494
Validation Loss : 0.9520

EPOCH | 7
Training Loss   : 0.7209
Validation Loss : 0.9338

E

Training Loss   : 0.4393
Validation Loss : 1.2610

EPOCH | 2
Training Loss   : 0.2805
Validation Loss : 1.8199

EPOCH | 3
Training Loss   : 0.2166
Validation Loss : 1.9053

EPOCH | 4
Training Loss   : 0.1451
Validation Loss : 2.0958

EPOCH | 5
Training Loss   : 0.1151
Validation Loss : 2.3503

EPOCH | 6
Training Loss   : 0.0823
Validation Loss : 2.4210

EPOCH | 7
Training Loss   : 0.0328
Validation Loss : 2.7631

EPOCH | 8
Training Loss   : 0.0072
Validation Loss : 2.8886

EPOCH | 9
Training Loss   : 0.0021
Validation Loss : 2.9254

EPOCH | 10
Training Loss   : 0.0014
Validation Loss : 2.9697

EPOCH | 11
Early stopping activated, with training and validation loss difference: 0.3159

Test number 68 | Start testing new parameter-combination...

Dataset loading...
Dataset loading completed

Model loading...
Model loading completed

Number of model parameters: 414625

EPOCH | 0
Training Loss   : 0.7307
Validation Loss : 0.9635

EPOCH | 1
Training Loss   : 0.6038
Validation Loss : 1.0462

E

Training Loss   : 0.0184
Validation Loss : 1.8907

EPOCH | 11
Early stopping activated, with training and validation loss difference: 0.0726

Test number 76 | Start testing new parameter-combination...

Dataset loading...
Dataset loading completed

Model loading...
Model loading completed

Number of model parameters: 414625

EPOCH | 0
Training Loss   : 0.4917
Validation Loss : 1.2189

EPOCH | 1
Training Loss   : 0.0335
Validation Loss : 1.7163

EPOCH | 2
Training Loss   : 0.0036
Validation Loss : 1.8870

EPOCH | 3
Training Loss   : 0.0019
Validation Loss : 1.9909

EPOCH | 4
Training Loss   : 0.0013
Validation Loss : 2.0686

EPOCH | 5
Training Loss   : 0.0010
Validation Loss : 2.1302

EPOCH | 6
Training Loss   : 0.0008
Validation Loss : 2.1823

EPOCH | 7
Training Loss   : 0.0007
Validation Loss : 2.2283

EPOCH | 8
Training Loss   : 0.0006
Validation Loss : 2.2705

EPOCH | 9
Training Loss   : 0.0005
Validation Loss : 2.3100

EPOCH | 10
Training Loss   : 0.0005
Validation Loss : 2.3466

E

Training Loss   : 0.7855
Validation Loss : 0.8938

EPOCH | 6
Training Loss   : 0.7793
Validation Loss : 0.8618

EPOCH | 7
Training Loss   : 0.7695
Validation Loss : 0.8676

EPOCH | 8
Training Loss   : 0.7557
Validation Loss : 0.8610

EPOCH | 9
Training Loss   : 0.7356
Validation Loss : 0.8873

EPOCH | 10
Training Loss   : 0.7269
Validation Loss : 0.9012

EPOCH | 11
Early stopping activated, with training and validation loss difference: 0.0303

Best hyperparameters were: {'batch_size': 16, 'learning_rate': 0.05, 'weight_decay': 0.001, 'pos_weight': 1.4590651175358011, 'model_hidden_units': 32, 'model_num_layers': 1, 'model_dropout': 0.0}



**Train with the best parameters**: The LSTM is trained using the selected parameters.

In [6]:
# Access the best parameters in order to train final model
with open('best_parameters_same_sets.txt') as f:
    data = f.read()

best_parameters_loaded = json.loads(data)

print('\nNow training with the best parameters\n')
training(params=best_parameters_loaded, make_err_logs=True)


Now training with the best parameters

Dataset loading...
Dataset loading completed

Model loading...
Model loading completed

Number of model parameters: 414625

EPOCH | 0
Training Loss   : 0.8279
Validation Loss : 0.8824

EPOCH | 1
Training Loss   : 0.8267
Validation Loss : 0.9247

EPOCH | 2
Training Loss   : 0.8188
Validation Loss : 0.8695

EPOCH | 3
Training Loss   : 0.8132
Validation Loss : 0.8670

EPOCH | 4
Training Loss   : 0.8113
Validation Loss : 0.8705

EPOCH | 5
Training Loss   : 0.8025
Validation Loss : 0.8510

EPOCH | 6
Training Loss   : 0.7873
Validation Loss : 0.8598

EPOCH | 7
Training Loss   : 0.7794
Validation Loss : 0.8590

EPOCH | 8
Training Loss   : 0.7558
Validation Loss : 0.8626

EPOCH | 9
Training Loss   : 0.7412
Validation Loss : 0.9003

EPOCH | 10
Training Loss   : 0.7115
Validation Loss : 0.8931

EPOCH | 11
Training Loss   : 0.6807
Validation Loss : 0.9478

EPOCH | 12
Training Loss   : 0.6327
Validation Loss : 0.9834

EPOCH | 13
Training Loss   : 0.5906
Vali

0.8509984329803703

The following Figure shows the ***training and validation set errors***, the epoch where the ***early stopping*** was activated, as well as the epoch of the final ***selected model***.

In [7]:
display(HTML('<img src="plots/train_valid_error.png?%d" height=400 width=400>' % __counter__))

**Test the model**: Predict the satisfiability of the clauses in the *Test Set* and get the corresponding metrics.

In [8]:
print('\nResults on the test set:\n')
testing(params=best_parameters_loaded)


Results on the test set:

Dataset loading...
Dataset loading completed

Model loading...

Model loading completed


Test set metrics:

 Confusion matrix: 
 [[233 152]
 [572 590]]
F1 Score  : 0.6197
Accuracy  : 0.5320
Precision : 0.5077
Recall    : 0.7951
ROC AUC   : 0.5423
Test Loss with final model : 0.8402215144068924


The following **Figures** show:
<ol>
<li>The <b>confusion matrix</b> for the test set-prediction</li>
<li>The <b>ROC-AUC curve</b> for the test set-prediction</li>
<li>The <b>precision recall curve</b> for the test set-prediction</li>
</ol>

In [9]:
print("\n1.")
display(HTML('<img src="plots/cm.png?%d" height=500 width=500>' % __counter__))
print("2.")
display(HTML('<img src="plots/roc_auc.png?%d" height=450 width=450>' % __counter__))
print("3.")
display(HTML('<img src="plots/pr.png?%d" height=450 width=450>' % __counter__))


1.


2.


3.


Since, as one can notice, ***the results are not very promising using this model and this architecture***, with the current amount of data, the model is not tested on a Test Set from a different data distribution.